# World Values Analysis

## 1. Define question <br>

This project focuses on understanding the differences between attitudes to work around the world, and what drives those differences. Specifically:  <br> <br>
a) What different values systems exist around the globe (focused on attitudes to work)? <br>
b) What socio-economic factors influence a country's values system? <br>
c) How well can we predict a country's values system, based on these factors?

## 2. Get data

### Import necessary modules & set environment variables

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from data_dictionary import *
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler

pd.set_option('display.max_columns', 500)

_random_state = 42

### Import data

In [ ]:
df = pd.read_stata('./world-values-survey-data/WVS_Longitudinal_1981_2014_stata_v2015_04_18.dta', convert_categoricals=False) # index_col='S025', 

### Clean data

#### Check shape and nulls; inspect header

In [ ]:
df.shape

In [ ]:
df.head()

#### We can relabel the columns using the data dictionary found at  http://www.worldvaluessurvey.org/WVSDocumentationWVL.jsp

In [ ]:
new_column_labels = []

for i in df.columns:
    new_column_labels.append(column_map_dictionary.get(i, i))

In [ ]:
new_column_labels2 = []

for i in new_column_labels:
    new_column_labels2.append(column_map_dictionary.get(i.lower(), i))

In [ ]:
df.columns = new_column_labels2

In [ ]:
df.head()

#### Note that the country is listed as a code (in 'Country/region' column). We want to add the name of the country

In [ ]:
# Turn each country/ code combination into an item in a list, by splitting on new lines
country_list = country_list.split('\n')

# Create a list of country codes and country names
country_code_list = []
country_name_list = []
for country_pair in country_list:
    country_code_list.append(country_pair.split(':')[0])
    country_name_list.append(country_pair.split(':')[1])
    
# Turn the country code / name lookup into a dataframe, to allow a merge with the original dataframe
country_dictionary = {'country_code': country_code_list, 'country_name': country_name_list}
country_lookup = pd.DataFrame(country_dictionary)

In [ ]:
# Merge country names into original dataframe
df['Country/region'] = df['Country/region'].astype(int) # match dtypes
country_lookup['country_code'] = country_lookup['country_code'].astype(int)
df = pd.merge(df, country_lookup, left_on='Country/region', right_on='country_code', how = 'left')
df.head()

In [ ]:
df['country_year'] = df['country_name'].astype(str) + '_' + df['Year survey'].astype(str)

#### Now, zoom in on the columns of interest:  What is important in a job?

In [ ]:
job_qs = important_in_a_job
baseline_qs = ['Wave', 'Year survey', 'Country/region', 'country_name', 'country_year']
job_df = df[baseline_qs + job_qs]

In [ ]:
job_df.head()

In [ ]:
# Count nulls
job_df.isnull().sum().sum()

In [ ]:
# Count values greater than or equal to zero (from the data dictionary, negative numbers indicate missing data)
(job_df[job_qs] >= 0).sum()

In [ ]:
# Focusing in on the columns with the fewest missing data points:
refined_job_qs = ['Important in a job: good pay',
 'Important in a job: not too much pressure',
 'Important in a job: good job security',
 'Important in a job: a respected job',
 'Important in a job: good hours',
 'Important in a job: an opportunity to use initiative',
 'Important in a job: generous holidays',
 'Important in a job: that you can achieve something',
 'Important in a job: a responsible job',
 'Important in a job: a job that is interesting',
 'Important in a job: a job that meets one´s abilities']

refined_job_df = df[baseline_qs + refined_job_qs].copy()

(refined_job_df[refined_job_qs] >= 0).sum()

In [ ]:
# Check how many people partially answered the questions
refined_job_df['number_of_jobs_qs_answered'] = (refined_job_df[refined_job_qs] >= 0).sum(axis = 1)

In [ ]:
refined_job_df['number_of_jobs_qs_answered'].value_counts()

In [ ]:
# Overall, most people answered all the questions (144k) or none (189k). 
# A small proportion (~6k) responsed to some but not all of the questions
# Dropping respondants who did not answer all questiosn will improve consistency and reduce bias, for a relatively small data loss
# Consequently, we will focus only on those respondants who answered all questions
mask = refined_job_df['number_of_jobs_qs_answered'] == 11
refined_job_df = refined_job_df[mask]

## 3. EDA

### Preview the data and look for correlations (clustering of input variables / multicolinearity)

In [ ]:
# View dataframe
refined_job_df.head()

In [ ]:
# Look at correlation matrix between different questions
sns.heatmap(refined_job_df[refined_job_qs].corr(), annot = True);

#### Though correlations are all < 0.5, we can see correlated clusters of variables:
- Achieving something, responsible job and opportunity to use initiative (correlations of ~0.4)
- Good hours, generous holidays, not too much pressure (correlations of ~0.35)

### Understand sample size

In [ ]:
# View dataframe
refined_job_df.head()

In [ ]:
# Summary statistics
country_counts = refined_job_df.groupby('country_year').count()
country_counts[refined_job_qs].describe()

In [ ]:
# Summary
print(f'Each question was, on average, answered by {country_counts.iloc[:,0].mean():.0f} people in each wave, with a \
min of {country_counts.iloc[:,0].min():.0f} and a max of {country_counts.iloc[:,0].max():.0f}')

### See how responses evolve on a country-by-country level over time

In [ ]:
# Aggregate
country_averages = refined_job_df.groupby('country_year').mean()
country_averages['country_year'] = country_averages.index
country_averages.head()

In [ ]:
# Add country name column
country_averages['country_name'] = [country_averages.index[i][0:country_averages.index[i].find('_')] for i in range(0, len(country_averages.index))]

In [ ]:
# Assess data availability
country_averages['country_name'].value_counts()[0:10]

In [ ]:
country_averages['country_name'].value_counts().count()

#### Of the 64 countries included in the datset, 8 have data from three or more waves; we will focus on these

In [ ]:
# Create sub-dfs for each of these countries
longitudinal_countries = ['Mexico', 'Japan', 'Argentina', 'India', 'China', 'Chile', 'Turkey','Spain']

In [ ]:
for index, df in country_averages.groupby(country_averages.index):
    if index == 'Mexico':
        display(df)

In [ ]:
# Plot evolution of values over time

country_averages.index = country_averages['country_name']

n_cols = 2
n_rows = int(len(longitudinal_countries)//n_cols)
figure, ax = plt.subplots(nrows = n_rows, ncols=n_cols, figsize=(18,36))
i = 0 # this is a counter to tell matplotlib which axis to plot on
_cmap = plt.get_cmap('tab20')

for index, dataframe in country_averages.groupby(country_averages.index):
    if index in longitudinal_countries:
        ax[i//n_cols][i%n_cols].set_title(f"\n {index} \n", fontsize = 15)
        for j in range(0, len(refined_job_qs)): # this for loop is used to improve line colors on charts
            ax[i//n_cols][i%n_cols].plot(dataframe['Year survey'], dataframe[refined_job_qs[j]], c=_cmap.colors[j])
        ax[i//n_cols][i%n_cols].set_xlabel('Year', horizontalalignment = 'right')
        ax[i//n_cols][i%n_cols].set_ylabel('Importance score')
        if i == 0:
            ax[i//n_cols][i%n_cols].legend(refined_job_qs, )
        i +=1

#### Normalizing our data

In some countries (e.g., Japan, Mexico) the variables tend to trend up and down together. This could be an artefact of how the survey was administered: the value for each cell is the percent of respondants who mentioned something as important. In some years, the interviews may have been longer or more comprehensive than others. To account for this fact, we can look at the number of times an attribute was mentioned as a % of the total.

In [ ]:
country_averages['total_mentions'] = country_averages[refined_job_qs].sum(axis = 1)
country_averages.head()

In [ ]:
for column in refined_job_qs:
    country_averages[f'normalized {column}']=country_averages[column]/country_averages['total_mentions'] * 100 
    # * 100 to get pct

In [ ]:
normalized_refined_job_qs = [f'normalized {column}'for column in refined_job_qs]
country_averages['normalized total_mentions'] = country_averages[normalized_refined_job_qs].sum(axis = 1)

In [ ]:
country_averages.head()

In [ ]:
# Plot evolution of values over time

country_averages.index = country_averages['country_name']

n_cols = 2
n_rows = int(len(longitudinal_countries)//n_cols)
figure, ax = plt.subplots(nrows = n_rows, ncols=n_cols, figsize=(18,30))
i = 0 # this is a counter to tell matplotlib which axis to plot on
_cmap = plt.get_cmap('tab20')

for index, dataframe in country_averages.groupby(country_averages.index):
    if index in longitudinal_countries:
        ax[i//n_cols][i%n_cols].set_title(f"\n {index} \n", fontsize = 15)
        for j in range(0, len(normalized_refined_job_qs)): # this for loop is used to improve line colors on charts
            ax[i//n_cols][i%n_cols].plot(dataframe['Year survey'], dataframe[normalized_refined_job_qs[j]], c=_cmap.colors[j])
        ax[i//n_cols][i%n_cols].set_xlabel('Year', horizontalalignment = 'right')
        # ax[i//n_cols][i%n_cols].set_xlim(xmin = 1980)
        ax[i//n_cols][i%n_cols].set_ylabel('Percent of total mentions')
        ax[i//n_cols][i%n_cols].legend(refined_job_qs, loc = 'lower left')
        i +=1

#### Feature engineering for clustering analysis

As noted above, we can see that certain variables are correlated:
- Achieving something, responsible job and opportunity to use initiative (correlations of ~0.4)
- Good hours, generous holidays, not too much pressure (correlations of ~0.35)

Using a methodology similar to PCA, we can create new features to reduce the dimensionality of our features.

In [ ]:
# Look at correlation matrix between different questions
sns.heatmap(country_averages[normalized_refined_job_qs].corr(), annot = True);

In [ ]:
# Good hours, not too much pressure, holidays
# Responsible, initiative, achieve
# Security, pay

In [ ]:
normalized_refined_job_qs

In [ ]:
# Create new features

# Achieving_responsible_initiative

country_averages['normalized achieving_responsible_initiative'] = \
    country_averages['normalized Important in a job: that you can achieve something'] + \
    country_averages['normalized Important in a job: a responsible job'] + \
    country_averages['normalized Important in a job: an opportunity to use initiative']

# Hours_holidays_pressure

country_averages['normalized hours_holidays_pressure'] = \
    country_averages['normalized Important in a job: good hours'] + \
    country_averages['normalized Important in a job: generous holidays'] + \
    country_averages['normalized Important in a job: not too much pressure']

In [ ]:
country_averages.head()

In [ ]:
country_averages.shape

In [ ]:
country_averages['normalized achieving_responsible_initiative'].mean()

In [ ]:
country_averages['normalized hours_holidays_pressure'].mean()

In [ ]:
country_averages[normalized_refined_job_qs].mean().sort_values(ascending=False)

Together, these two features account for ~45% of the variance. "Good pay" and "good job security" are also important, accounting for ~14% and ~12% of responses respectively. In addition, they are correlated (0.32) so we'll try engineering a new feature combining both of these.

In [ ]:
# Security_pay

country_averages['normalized security_pay'] = \
    country_averages['normalized Important in a job: good job security'] + \
    country_averages['normalized Important in a job: good pay']

In [ ]:
country_averages['normalized security_pay'].mean()

In [ ]:
engineered_job_attributes = ['normalized achieving_responsible_initiative', 
                             'normalized hours_holidays_pressure', 
                             'normalized security_pay']

Now, our three engineered features account for ~75% of responses, using only three features vs. the original eleven. While this is not as mathematically rigorous as PCA (in terms of maximizing variance captured), creating meaningful combinations of variables helps us maintain interpretability.

#### Clustering analysis

In [ ]:
# Write a plotter function

def plotter(df, input_cols, \
            k_means_n_clusters = 3, agglom_n_clusters = 3, \
            dbscan_eps = 0.5, dbscan_min_samples = 5, n_rows = 1, n_cols = 3, _cmap = plt.get_cmap('tab20')):
    '''function to visualize kmean, agglomerative and dbscan clustering
    args: dataframe, input_cols --> to pull in from df, 
    kwargs: k_means_n_clusters, agglom_n_clusters, dbscan_eps, dbscan_min_samples, n_rows, n_cols --> rows/ cols of subplots, _cmap'''
    # pull out columns to plot and scale
    cols_to_plot = df[input_cols]
    figure, ax = plt.subplots(nrows = n_rows, ncols=n_cols, figsize=(18,5))
    # k means
    kmeans = KMeans(n_clusters=k_means_n_clusters)
    kmeans.fit(cols_to_plot)
    df['kmeans_labels'] = kmeans.labels_
    for index, sub_dataframe in df.groupby('kmeans_labels'):
        ax[0].set_title("K means")
        ax[0].scatter(sub_dataframe[input_cols[0]], sub_dataframe[input_cols[1]], c=_cmap.colors[index], label = f"Index {index}")
        ax[0].set_xlabel(input_cols[0])
        ax[0].set_ylabel(input_cols[1])
        ax[0].legend()
    # agglom
    agglom = AgglomerativeClustering(n_clusters=agglom_n_clusters)
    agglom.fit(cols_to_plot)
    df['agglom_labels'] = agglom.labels_
    for index, sub_dataframe in df.groupby('agglom_labels'):
        ax[1].set_title("Agglomerative clustering")
        ax[1].scatter(sub_dataframe[input_cols[0]], sub_dataframe[input_cols[1]], c=_cmap.colors[index], label = f"Index {index}")
        ax[1].set_xlabel(input_cols[0])
        ax[1].set_ylabel(input_cols[1])
        ax[1].legend()
    # dbscan
    dbmodel = DBSCAN(eps = dbscan_eps, min_samples=dbscan_min_samples)
    dbmodel.fit(cols_to_plot)
    df['dbscan_labels'] = dbmodel.labels_
    for index, sub_dataframe in df.groupby('dbscan_labels'):
        ax[2].set_title("DBSCAN")
        ax[2].scatter(sub_dataframe[input_cols[0]], sub_dataframe[input_cols[1]], c=_cmap.colors[index], label = f"Index {index}")
        ax[2].set_xlabel(input_cols[0])
        ax[2].set_ylabel(input_cols[1])
        ax[2].legend()

In [ ]:
# Plot countries, using different clustering algorithms and variables
plotter(df = country_averages, 
        input_cols = ['normalized achieving_responsible_initiative', 'normalized hours_holidays_pressure'],
        k_means_n_clusters = 3, agglom_n_clusters = 3, dbscan_eps = 2)

In [ ]:
plotter(df = country_averages, 
        input_cols = ['normalized achieving_responsible_initiative', 'normalized security_pay'],
        k_means_n_clusters = 3, agglom_n_clusters = 3, dbscan_eps = 2)

In [ ]:
plotter(df = country_averages, 
        input_cols = ['normalized hours_holidays_pressure', 'normalized security_pay'],
        k_means_n_clusters = 3, agglom_n_clusters = 3, dbscan_eps = 2)

#### The top left chart (kmeans clustering with the two axes as achieving/ responsible/ initiative and hours/ holidays/ pressure accounts) provides the best visual separation. Proceeding with this...

In [ ]:
# Modify our plotter to focus on k-means and add point labels

def k_means_plotter(df, input_cols, k_means_n_clusters = 3, n_rows = 1, n_cols = 1, _cmap = plt.get_cmap('tab20')):
    '''function to visualize kmean clustering
    args: dataframe, input_cols --> to pull in from df, agglom_n_clusters, dbscan_eps, dbscan_min_samples
    kwargs: k_means_n_clusters = 3, n_rows = 1, n_cols =1 --> rows/ cols of subplots, _cmap = plt.get_cmap('tab20')'''
    # pull out columns to plot and scale
    cols_to_plot = df[input_cols]
    figure, ax = plt.subplots(nrows = n_rows, ncols=n_cols, figsize=(18,11))
    # k means
    kmeans = KMeans(n_clusters=k_means_n_clusters, random_state=_random_state)
    kmeans.fit(cols_to_plot)
    df['kmeans_labels'] = kmeans.labels_
    for index, sub_dataframe in df.groupby('kmeans_labels'):
        ax.set_title("\n Country value systems\n ", size = 18)
        ax.scatter(sub_dataframe[input_cols[0]], sub_dataframe[input_cols[1]], c=_cmap.colors[index], label = f"Index {index}")
        ax.set_xlabel('% of responses focused on achievement', size = 14)
        ax.tick_params(labelsize = 14)
        #ax.set_xlim(xmax = 33)
        ax.set_ylabel('% of responses focused on lifestyle', size = 14)
        for j, k in sub_dataframe['country_year'].iteritems():
            x = sub_dataframe[input_cols[0]][j]+.1
            y = sub_dataframe[input_cols[1]][j]+.05
            label = k
            ax.annotate(label, (x, y), rotation= 20, ha = 'left', va = 'bottom', size = 10)
        ax.legend(["Balanced","Achievement","Lifestyle"], fontsize = 12)

In [ ]:
# Reset index to unique values
country_averages.index = country_averages['country_year']

In [ ]:
k_means_plotter(df = country_averages, 
        input_cols = ['normalized achieving_responsible_initiative', 'normalized hours_holidays_pressure'])

#### The clustering analysis identifies three groups:
- **The lifestyle group**: Countries where people are less concerned with having a high-achieving job and moderately concerned about lifestyle (vacations, hours, pressure) - including Russia, Albania and Slovakia
- **The achievement group**: Countries where people want a high-achieving job and care less about lifestyle - including Norway, Sweden and Australia
- **The balanced group**: Countries that balance achievement with lifestyle - including Turkey, Morocco and South Korea

#### Next, we will use multi-class logistic regression to see if we can predict a country's cluster based on economic variables.

#### Two notes:
- The boundaries between the groups are somewhat fuzzy (e.g., the U.S. in 1990 could also be assigned to the high-achievement group
- Many countries (for example Japan and Turkey) are fairly stable in values over time. Building on this analysis, we could look at what drives changes.

In [ ]:
# Merge in labels

cluster_labels_df = pd.DataFrame([{'kmeans_labels': 1, 'kmeans_value_cluster' :'achievement'},
                                  {'kmeans_labels': 2, 'kmeans_value_cluster' :'lifestyle'},
                                  {'kmeans_labels': 0, 'kmeans_value_cluster' :'balanced'}])

country_averages = pd.merge(country_averages, cluster_labels_df, on = 'kmeans_labels')
country_averages.index = country_averages['country_year']

In [ ]:
country_averages.head()

In [ ]:
# Confirm value clusters correctly mapped by looking at one example from each cluster
balanced_check = country_averages.loc['Uganda_2001','kmeans_value_cluster'] # balanced cluster
achievement_check = country_averages.loc['Peru_1996','kmeans_value_cluster'] # achievement cluster
lifestyle_check = country_averages.loc['Finland_1981','kmeans_value_cluster'] # lifestyle cluster

balanced_check, achievement_check, lifestyle_check

In [ ]:
country_averages.shape

### Logistic regression

#### Merge in indicator data - GDP

In [ ]:
# Read in dataframe (from https://data.worldbank.org/, accessed July 2018)
gdp_df = pd.read_csv('./indicator_data/World_bank_GDP_per_cap_PPP_df.csv')

In [ ]:
gdp_df.head()

In [ ]:
# Check shape of dataframe
gdp_df.shape

In [ ]:
# Check percent nulls
print(f'{gdp_df.isnull().sum().sum()*100/(gdp_df.shape[0]*gdp_df.shape[1]):.0f}% null')

In [ ]:
# The country_averages df has a two-level index (country_year). We need to stack our gdp dataframe to match

In [ ]:
gdp_df.index = gdp_df['Country Name']
gdp_df.drop('Country Name', axis = 1, inplace = True)

gdp_df = pd.DataFrame(gdp_df.stack(), columns=["GDP per capita, PPP"])

gdp_df.reset_index(inplace=True)

gdp_df.index = gdp_df['Country Name'] + "_" + gdp_df['level_1']

gdp_df.drop(['Country Name', 'level_1'], axis = 1, inplace= True)

gdp_df.head()

In [ ]:
country_averages.head()

In [ ]:
# Check our country_averages df has no nulls
assert country_averages.isnull().sum().sum() == 0

In [ ]:
# Merge in GDP information

In [ ]:
job_and_gdp_df = pd.merge(country_averages, gdp_df, how = 'left', left_index = True, right_index= True)

In [ ]:
# Preview new dataframe

In [ ]:
job_and_gdp_df.head()

**Assess introduction of null values**

We had introduced 30 nulls, out of 104 values. Refering to the null values using "job_and_gdp_df.sort_values('GDP per capita, PPP', na_position='first').head(15)" we can fix many of these by matching our country names to country names in the World Bank database (e.g., Dominican Republic vs. Dominican Rep.). Post-processing, 11 nulls remain. These are mostly due to timing, as the World Values Survey was administered in a few countries before 1990 whereas the World Bank PPP GDP figures begin in 1990.

In [ ]:
job_and_gdp_df['Wave'].count(), job_and_gdp_df['GDP per capita, PPP'].isnull().sum()

We will drop the nulls from our dataframe going forwards. For easy reference, we will save the dataframe with nulls as job_and_gdp_df_original.

In [ ]:
job_and_gdp_df_original = job_and_gdp_df
job_and_gdp_df = job_and_gdp_df.dropna()

In [ ]:
job_and_gdp_df.shape, job_and_gdp_df_original.shape

#### Merge in indicator data - GINI

In [ ]:
# Read in dataframe (from https://data.worldbank.org/, accessed July 2018)
gini_df = pd.read_csv('./indicator_data/gini.csv')

In [ ]:
# Preview data
gini_df.head()

In [ ]:
# Impute nulls
gini_df.index = gini_df['Country Name']
gini_df.drop('Country Name', axis = 1, inplace=True)
gini_df.fillna(method = 'ffill', axis=1, inplace=True)         # first try a forward fill (i.e., base on the last available value)
gini_df.fillna(method = 'backfill', axis=1, inplace= True)     # next try a backward fill (i.e., base on the next available value)
gini_df.fillna(value = gini_df.mean(), inplace=True)           # where no data is available fill with the dataframe mean

In [ ]:
gini_df.head()

In [ ]:
# The country_averages df has a two-level index (country_year). We need to stack our gini dataframe to match

In [ ]:
gini_df = pd.DataFrame(gini_df.stack(), columns=["GINI coefficient"])

gini_df.reset_index(inplace=True)

gini_df.index = gini_df['Country Name'] + "_" + gini_df['level_1']

gini_df.drop(['Country Name', 'level_1'], axis = 1, inplace= True)

gini_df.head()

In [ ]:
# Check our country_averages df has no nulls
assert country_averages.isnull().sum().sum() == 0

In [ ]:
# Merge in GINI information

job_and_indicator_df = pd.merge(job_and_gdp_df, gini_df, how = 'left', left_index = True, right_index= True)

In [ ]:
# Preview new dataframe

job_and_indicator_df.head()

**Assess introduction of null values**

In [ ]:
job_and_indicator_df.isnull().sum().sum()

- We had introduced 22 nulls, out of 93 values. We can fix many of these by matching our country names to country names in the World Bank database (e.g., Dominican Republic vs. Dominican Rep.). 
- Post-processing, 5 nulls remained. These are due to a handful of countries that do not report GINI scores (New Zealand, Puerto Rico, Saudi Arabia, Singapore)
- These remaining nulls were imputed with the average GINI coeff across all countries, leading to no additional nulls being inserted into our data

#### Merge in indicator data - primary school completion

In [ ]:
# Read in dataframe (from https://data.worldbank.org/, accessed July 2018)
primary_completion_df = pd.read_csv('./indicator_data/primary_completion.csv')

In [ ]:
# Preview data
primary_completion_df.head()

In [ ]:
# Impute nulls
primary_completion_df.index = primary_completion_df['Country Name']
primary_completion_df.drop('Country Name', axis = 1, inplace=True)
primary_completion_df.fillna(method = 'ffill', axis=1, inplace=True)         # first try a forward fill (i.e., base on the last available value)
primary_completion_df.fillna(method = 'backfill', axis=1, inplace= True)     # next try a backward fill (i.e., base on the next available value)

primary_completion_df.fillna(value = 100, inplace=True)           # three countries have no data available for any years:
# the United States, Australia and Bosnia. Since primary education is compulsory and free in these countries,
# we impute a value of close to 100%

In [ ]:
primary_completion_df.head()

In [ ]:
# The country_averages df has a two-level index (country_year). We need to stack our education dataframe to match

In [ ]:
primary_completion_df = pd.DataFrame(primary_completion_df.stack(), columns=["Primary completion rate"])

primary_completion_df.reset_index(inplace=True)

primary_completion_df.index = primary_completion_df['Country Name'] + "_" + primary_completion_df['level_1']

primary_completion_df.drop(['Country Name', 'level_1'], axis = 1, inplace= True)

primary_completion_df.head()

In [ ]:
# Check our country_averages df has no nulls
assert job_and_indicator_df.isnull().sum().sum() == 0

In [ ]:
# Merge in GINI information

job_and_indicator_df = pd.merge(job_and_indicator_df, primary_completion_df, how = 'left', left_index = True, right_index= True)

In [ ]:
# Preview new dataframe

job_and_indicator_df.head()

**Assess introduction of null values**

In [ ]:
job_and_indicator_df.isnull().sum().sum()

- We had introduced 53 nulls, out of 93 values. We can fix many of these by matching our country names to country names in the World Bank database (e.g., Dominican Republic vs. Dominican Rep.). 
- Post-processing, 43 nulls remained
- Using back and forward fill to impute missing values, we can reduce the number of nulls to 5
- These are due to a handful of countries that do not report primary school completion (Australia, Bosnia, United States)
- Primary education is mandatory and free for children in all these countries, so we can impute a completion rate of close to 100

#### Merge in indicator data - democracy index
Overall polity score from the Polity IV dataset, calculated by subtracting an autocracy score from a democracy score. It is a summary measure of a country's democratic and free nature. -10 is the lowest value, 10 the highest.

In [ ]:
# Read in dataframe (from https://www.gapminder.org/data/ ('Democracy Index'), accessed July 2018)
democracy_df = pd.read_csv("./indicator_data/Democracy_index_gapminder.csv")

In [ ]:
# Preview data
democracy_df.head()

In [ ]:
# Example country: UK
# democracy_df.index = democracy_df['Democracy index']
# UK = pd.DataFrame(democracy_df.loc['United Kingdom', :])
# UK.T

In [ ]:
# Impute nulls
democracy_df.rename(columns = {'Democracy index':'Country Name'}, inplace=True)   # rename our country labels to match prior dataframes
democracy_df.index = democracy_df['Country Name']
democracy_df.drop('Country Name', axis = 1, inplace=True)
democracy_df.fillna(method = 'ffill', axis=1, inplace=True)                     # first try a forward fill (i.e., base on the last available value)
democracy_df.fillna(method = 'backfill', axis=1, inplace= True)                 # next try a backward fill (i.e., base on the next available value)

democracy_df.fillna(value = democracy_df.mean(), inplace=True)

In [ ]:
# The country_averages df has a two-level index (country_year). We need to stack our democracy dataframe to match

In [ ]:
democracy_df = pd.DataFrame(democracy_df.stack(), columns=["Democracy index"])

democracy_df.reset_index(inplace=True)

democracy_df.index = democracy_df['Country Name'] + "_" + democracy_df['level_1']

democracy_df.drop(['Country Name', 'level_1'], axis = 1, inplace= True)

democracy_df.head()

In [ ]:
# Check our country_averages df has no nulls
assert job_and_indicator_df.isnull().sum().sum() == 0

In [ ]:
# Merge in democracy information

job_and_indicator_df = pd.merge(job_and_indicator_df, democracy_df, how = 'left', left_index = True, right_index= True)

In [ ]:
# Preview new dataframe

job_and_indicator_df.head()

**Assess introduction of null values**

In [ ]:
job_and_indicator_df.isnull().sum().sum()

In [ ]:
# job_and_indicator_df.sort_values('Democracy index', na_position='first')

- We had introduced 10 nulls, out of 93 values. We can fix many of these by matching our country names to country names in the World Bank database (e.g., Dominican Republic vs. Dominican Rep.). 
- Post-processing, 5 nulls remained
- Using back and forward fill to impute missing values, we can reduce the number of nulls to 2
- There is no score for Puerto Rico, but we can impute this with the mean for the dataframe

#### Create baseline for model

We have identified three clusters of attitudes towards jobs - the lifestyle group, the achievement group and the balanced group. Our model will be a logistic regression model to assess whether we can predict which cluster a country is in at a given point in time, based on its GDP.

The simplest model we could create is to predict the majority class each time

In [ ]:
# Identifying the majority class
job_and_gdp_df['kmeans_labels'].value_counts()

In [ ]:
# Baseline accuracy (i.e., the accuracy of a model that just predicts the majority class)
print(f'Baseline accuracy = {max(job_and_gdp_df["kmeans_labels"].value_counts())/ job_and_gdp_df.shape[0]*100:.0f}%')

#### Normalize features to allow for feature importance selection

In [ ]:
indicator_features = ['GDP per capita, PPP', 'GINI coefficient', 'Primary completion rate', 'Democracy index']

In [ ]:
ss = StandardScaler()

In [ ]:
scaled_indicator_columns = pd.DataFrame(ss.fit_transform(job_and_indicator_df[indicator_features]),
                                       columns = ['Scaled GDP per capita, PPP', 'Scaled GINI coefficient', 'Scaled Primary completion rate', 'Scaled Democracy index'],
                                       index = job_and_indicator_df.index)

In [ ]:
job_and_indicator_df = pd.merge(job_and_indicator_df, scaled_indicator_columns, left_index=True, right_index=True)

In [ ]:
job_and_indicator_df.head()

#### Train-test split

In [ ]:
# Determine test size
job_and_indicator_df.shape

We have 93 rows of data. Using a 80/20% train test split gives us ~75 rows to train our model and ~18 to test

In [ ]:
# For a linear regression, we need our data to be ordinal categories (vs. disparate categories)
# We can roughly order them by making 'lifestyle' -1, 'balanced' 0 and 'achievement' 1

ordinal_mapping = pd.DataFrame([{'kmeans_value_cluster':'lifestyle','ordinal_kmeans_label': -1},
                  {'kmeans_value_cluster':'balanced','ordinal_kmeans_label': 0},
                  {'kmeans_value_cluster':'achievement','ordinal_kmeans_label': 1}])

job_and_indicator_df = pd.merge(job_and_indicator_df, ordinal_mapping, on = 'kmeans_value_cluster')
job_and_indicator_df.index = job_and_indicator_df['country_year']

job_and_indicator_df.head()

In [ ]:
feature_list = [i for i in job_and_indicator_df.columns if i !='ordinal_kmeans_label']
target = 'ordinal_kmeans_label'
train_X, test_X, train_y, test_y = train_test_split(job_and_indicator_df[feature_list], job_and_indicator_df[target], test_size = 0.2)

#### Build linear regression model to predict attitudes towards jobs from development indicator values

In [ ]:
lr_achievement_axis = LinearRegression()
lr_achievement_axis.fit(train_X[indicator_features], train_X['normalized achieving_responsible_initiative'])
lr_achievement_axis.score(test_X[indicator_features], test_X['normalized achieving_responsible_initiative'])

In [ ]:
lr_lifestyle_axis = LinearRegression()
lr_lifestyle_axis.fit(train_X [indicator_features], train_X ['normalized hours_holidays_pressure'])
lr_lifestyle_axis.score(test_X[indicator_features], test_X ['normalized hours_holidays_pressure'])

In [ ]:
lr_income_security_axis = LinearRegression()
lr_income_security_axis.fit(train_X[indicator_features], train_X['normalized security_pay'])
lr_income_security_axis.score(test_X[indicator_features], test_X['normalized security_pay'])

#### Add in cross-validation

In [ ]:
# Set number of folds for cross validation. We use CV = 10. While higher than a typical value of 3-5,  this will 
# allow us to use a higher portion of our dataset to train the model which is useful as we have a small dataset
_cv = 10 

In [ ]:
indicator_features

In [ ]:
for i in engineered_job_attributes:
    cvs = cross_val_score(LinearRegression(),
                                       train_X[indicator_features], 
                                       train_X[i], 
                                       cv = _cv)
    print(f'\n The mean and range of R2 values for **{i}** are {cvs.mean():.2f} and {cvs.min():.2f} - \
{cvs.max():.2f}. The values are {list(np.round(cvs, 2))}.')

*Overall, we see that our features are fairly good at predicting the achievement axis, but less successful at predicting the lifestyle axis (holiday/hours/pressure) and income security axis (security/pay). Our clustering analysis builds on both the achievement and lifestyle axes, so our logistic regression will be able to provide some prediction of cluster based on the achievement axis but will struggle on the lifestyle axis. One possible build is to add more features that better predict the lifestyle axis.*

#### Update data: use normalized data to allow feature importance analysis

In [ ]:
for i in engineered_job_attributes:
    cvs = cross_val_score(LinearRegression(),
                                       train_X[i].values.reshape(-1,1), 
                                       train_y, 
                                       cv = _cv)
    print(f'\n The mean and range of R2 values for **{i}** are {cvs.mean():.2f} and {cvs.min():.2f} - \
{cvs.max():.2f}. The values are {list(np.round(cvs, 2))}.')

*We want to understand which features are most important in predicting attitudes towards jobs. This section will focus coefficient interpretation the achivement axis, as our model is most effective for that axis. (Later, we'll map in more data to get a better prediction on the lifestyle axis. For now we'll ignore the income security axis, as that's not used in our clustering analysis)*

In [ ]:
scaled_indicators = ['Scaled GDP per capita, PPP', 'Scaled GINI coefficient',
       'Scaled Primary completion rate', 'Scaled Democracy index']
cvs_achievement_axis_normalized = cross_val_score(LinearRegression(), 
                                                  train_X[scaled_indicators], train_y, cv = _cv)

In [ ]:
cvs_achievement_axis_normalized.mean()

#### Map in more features that could be used to predict the 'lifestyle' axis

In [ ]:
# cell subscriptions - https://data.worldbank.org/indicator/IT.CEL.SETS.P2?end=2016&start=1995&view=chart
# life expectancy - https://data.worldbank.org/indicator/SP.DYN.LE00.FE.IN?view=chart
# employment in ag - https://data.worldbank.org/indicator/SL.AGR.EMPL.MA.ZS?view=chart
# employment in industry - https://data.worldbank.org/indicator/SL.IND.EMPL.MA.ZS?view=chart
# employment in services - https://data.worldbank.org/indicator/SL.SRV.EMPL.MA.ZS?view=chart

In [ ]:
cell_subs_df = pd.read_csv("./indicator_data/cell_subscriptions.csv")
life_expectancy_female_df = pd.read_csv("./indicator_data/Life expectancy female.csv")
primary_employment_male_df = pd.read_csv("./indicator_data/Ag employment pct male.csv")
secondary_employment_male_df = pd.read_csv("./indicator_data/Industry employment pct male.csv")
tertiary_employment_male_df = pd.read_csv("./indicator_data/Services employment pct male.csv")

In [ ]:
# Impute nulls

new_indicator_dfs = [cell_subs_df, life_expectancy_female_df, 
                     primary_employment_male_df, secondary_employment_male_df, tertiary_employment_male_df]

for i in range(0, len(new_indicator_dfs)):
    new_indicator_dfs[i].index = new_indicator_dfs[i]['Country Name']
    new_indicator_dfs[i].drop('Country Name', axis = 1, inplace=True)
    new_indicator_dfs[i] = new_indicator_dfs[i].fillna(method = 'ffill', axis=1)             # first try a forward fill (i.e., base on the last available value)
    new_indicator_dfs[i] = new_indicator_dfs[i].fillna(method = 'backfill', axis=1)          # next try a backward fill (i.e., base on the next available value)
    new_indicator_dfs[i] = new_indicator_dfs[i].fillna(value = new_indicator_dfs[i].mean())  # finally impute with df mean

#### Build logistic regression model

#### Evaluate model